### HCP Performance Evaluation ###

In [ ]:
import os
import numpy as np
import pandas as pd
from typing import List

# Binary perfomance metrics
from sklearn.metrics import confusion_matrix

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.performance import binary_performance

In [ ]:
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
results_file_name = 'predict_mental_health_250414.parquet'
results_file = os.path.join(data_dir, results_file_name)
df = pd.read_parquet(results_file)
display(df.head(2))

In [ ]:
# Select the predicted variables and ground truth rows
var_col = 'mental_health'
pred_col = 'predict_mh'
pred_score_col = 'predict_mh_score'

var_cols = ['id', 'name', 'description', var_col, pred_col, pred_score_col]
pred_vals = [0, 1]
df_var = df.loc[df[var_col].isin(pred_vals)][var_cols].\
                reset_index(drop=True)
print(df_var.shape)
display(df_var.head(2))

y_true = list(df_var[var_col].values)
y_pred = list(df_var[pred_col].values)
y_pred = [1 if pred else 0 for pred in y_pred]
y_true = [int(y) for y in y_true]

### Performance metrics manual calculation ###

In [ ]:
# Original performance
performance_dict = binary_performance(y_true=y_true, y_pred=y_pred)
display(performance_dict)
# Performance if we set all prediction to 1 
print()
performance_dict_1 = binary_performance(y_true=y_true, y_pred=np.ones(len(y_pred)))
display(performance_dict_1)

### Evaluate false positives ###

In [ ]:
fp_idx = [idx for idx in range(len(y_true)) if (y_true[idx] == 0) & (y_pred[idx] == 1)]
fp_id = df_var.loc[df_var.index.isin(fp_idx), 'id'].values

print(fp_idx)

for company_id in fp_id[:2]:
    df_company = df_var.loc[df_var['id'] == company_id]
    display(df_company)
    print(df_company['name'].values[0])
    print(df_company['description'].values[0])
    print()